In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... d

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec, FastText

from autogluon.tabular import TabularPredictor, TabularDataset
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import pandas as pd
import numpy as np

from copy import deepcopy
from collections import Counter
import pytz
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 임베딩 모델 설정 및 전처리 옵션

In [17]:
# embedding_model = 'w2v'
embedding_model = 'fast'
pattern = r'[\s\-_\(\)\/]'
full_name =  {
'ais': 'alarm indication signal',
'batt': 'battery',
'cep': 'critical event prevention',
'conn': 'connection',
'crc': 'cyclic redundancy check',
'csf': 'client signal fail',
'dcc': 'data communication channel',
'dcn': 'data communication network',
'dc': 'direct current',
'ddm': 'digital diagnostic monitoring',
'env': 'environment',
'err': 'error',
'eth': 'ethernet',
'fan': 'fan',
'fl': 'frame loss',
'idf': 'in-service data frame',
'init': 'initialization',
'ioc': 'input output controller',
'ipc': 'inter process communication',
'iof': 'ingress only filtering',
'll': 'link-layer',
'llcf': 'link level control field',
'loc': 'loss of communication',
'loc': 'loss of continuity',
'lof': 'loss of frame',
'los': 'loss of signal',
'lsp': 'label switched path',
'meg': 'mega',
'mep': 'maintenance entity group end point',
'mis': 'misalignment',
'ne': 'network element',
'nvram': 'non volatile random access memory',
'oam': 'operations, administration, and maintenance',
'oamloss': 'operations administration and maintenance loss',
'odi': 'optical delay interferometer',
'opt': 'optical',
'pde': 'path delay estimation',
'pdm': 'physical data model',
'poam': 'physical operations, administration, and maintenance',
'ps': 'packet switching',
'psm': 'power save mode',
'psu': 'power supply unit',
'pw': 'pseudowire',
'pwe': 'pseudowire emulation',
'ql': 'queue length',
'rdi': 'remote defect indication',
'rem': 'remote',
'rfa': 'remote failure analysis',
'rmt': 'remote',
'rx': 'receiver',
'rsmt': 'required signal-to-noise ratio margin threshold',
'sfp': 'small form factor pluggable',
'stm': 'synchronous transport module',
'sys': 'system',
'tca': 'threshold crossing alert',
'tdm': 'time division multiplexing',
'trk': 'track',
'tx': 'transmit',
'urc': 'unidirectional remote control',
}

# 데이터 불러오기

In [5]:
train_path = '/content/drive/MyDrive/my_data/kt_network/q2/Q2_train.csv'
test_path = '/content/drive/MyDrive/my_data/kt_network/q2/Q2_test.csv'
submisson_path = '/content/drive/MyDrive/my_data/kt_network/q2/Q2_label_sample.csv'
save_path = '/content/drive/MyDrive/kt_network_competition/AutogluonModels_kt_network'
w2v_path = '/content/drive/MyDrive/kt_network_competition/w2v_model'
fast_path = '/content/drive/MyDrive/kt_network_competition/fasttext_model'

In [6]:
target = "root_cause_type"

data = pd.read_csv('/content/drive/MyDrive/my_data/kt_network/q2/Q2_train.csv')
data

,ticketno,alarmno,alarmtime,alarmlevel,alarmmsg_original,site,sysname,unit,slot,port,sva,root_cause_domain,root_cause_type
0,21122633.0,1669820428245,2022-12-01 00:02:24+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
1,21122633.0,1669821318728,2022-12-01 00:17:15+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
2,21122633.0,1669822214832,2022-12-01 00:32:11+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
3,21122633.0,1669823114128,2022-12-01 00:47:10+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
4,21122633.0,1669824028082,2022-12-01 01:02:24+09:00,5,ETH-ERR,ACEN,acnt,EQPT,3.0,1.0,NSA,A,LinkCut
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9317,21774618.0,1671892499215,2022-12-24 23:37:14+09:00,7,OPT-LOS,AFAA,agow,EQPT,5.0,5.0,NSA,A,LinkCut
9318,15693425.0,877949375,2022-12-25 10:13:46+09:00,7,OPT-REMOVE,AGFD,aibb,EQPT,1.0,8.0,NSA,A,UnitFail
9319,21809789.0,1671974758375,2022-12-25 22:28:14+09:00,7,OPT-LOS,ADKA,aeaq,EQPT,3.0,3.0,NSA,A,LinkCut
9320,21811213.0,1671978167736,2022-12-25 23:25:03+09:00,7,OPT-LOS,ABZO,acie,EQPT,5.0,6.0,NSA,A,LinkCut


In [7]:
test = pd.read_csv("/content/drive/MyDrive/my_data/kt_network/q2/Q2_test.csv")
test

,ticketno,alarmno,alarmtime,alarmlevel,alarmmsg_original,site,sysname,unit,slot,port,sva,root_cause_domain
0,21812391.0,1671894138838,2022-12-25 00:02:16+09:00,5,Loss Of Signal,AEAQ,afbd,r1sr1sl16/ETHLocPort#15#1,16.0,15.0,NSA,C
1,21775988.0,1671894172511,2022-12-25 00:02:51+09:00,5,Loss Of Signal,ADZW,aezn,r1sr1sl13/ETHLocPort#20#1,13.0,20.0,NSA,C
2,21792259.0,1671894204682,2022-12-25 00:03:22+09:00,4,DDM_RX_PWR_HIGH,AECE,afeg,X2FUA,2.0,1.0,NSA,B
3,21812412.0,1671894215702,2022-12-25 00:03:33+09:00,5,BATT_ENV_FAIL,ACCN,aclp,---,NaN,NaN,SA,B
4,21812417.0,1671894220812,2022-12-25 00:03:39+09:00,5,Loss Of Signal,AEAQ,afbd,r1sr1sl16/ETHLocPort#16#1,16.0,16.0,NSA,C
...,...,...,...,...,...,...,...,...,...,...,...,...
37666,21986223.0,1672412311698,2022-12-30 23:58:30+09:00,5,Loss Of Signal,ADZW,aezn,r1sr1sl13/ETHLocPort#20#1,13.0,20.0,NSA,C
37667,22015278.0,1672412316271,2022-12-30 23:58:33+09:00,4,DDM_RX_PWR_HIGH,AEMD,afsr,G16FU,5.0,6.0,SA,B
37668,21986426.0,1672412317238,2022-12-30 23:58:34+09:00,5,MEP_LSP_RDI,ACMY,acxj,G2FUA,1.0,1.0,NSA,B
37669,22015300.0,1672412373531,2022-12-30 23:59:32+09:00,5,Loss Of Signal,AEAQ,afbd,r1sr1sl16/ETHLocPort#16#1,16.0,16.0,NSA,C


# 단어 임베딩 모델 불러오기

In [8]:
# 모델 생성
if embedding_model == 'w2v':
    model = Word2Vec.load(w2v_path)
elif embedding_model == 'fast':
    model = FastText.load(fast_path)
# 단어를 임베딩한 벡터 값 확인
print(model.wv['communication', 'data', 'network'])

[[ 2.33462139e-04  1.10550527e-03 -8.30097066e-04 -9.65214160e-04
   1.65458478e-03  1.84627905e-04  1.73240493e-03 -2.34158098e-04
  -2.08324310e-03 -1.01679834e-04  2.85268109e-03  7.06844672e-04
   1.54870690e-03  2.28503501e-04  9.95156239e-04  9.44359053e-04
  -1.14503561e-03  4.91483370e-04 -9.21080296e-04  1.61688324e-04
   6.10307325e-04  6.01704989e-04  3.35507822e-04 -9.15470295e-08
  -4.91310318e-04 -8.73949553e-04 -1.35218864e-03  1.55750095e-04
  -1.55129033e-04 -8.36733088e-05  1.01745839e-03  7.60567491e-04
  -2.55621108e-03 -6.10819203e-04  4.00126482e-05 -1.01661379e-03
   2.64662970e-03  1.94886758e-04 -5.50074619e-04  6.32610230e-04
  -9.65014915e-04 -8.58902989e-04 -1.65244425e-03  1.20835635e-03
   1.42975070e-03 -1.72063825e-03  7.63372867e-04 -9.44979955e-04
  -5.51469275e-05 -9.46307147e-04  2.99201114e-03 -7.33090856e-04
  -1.73058652e-04 -7.49957835e-05  2.86441045e-05 -2.30262452e-03
  -3.40513740e-04  5.78511972e-04  1.29883550e-03  2.06581061e-03
   1.45014

# 전처리

In [26]:
# 축약어를 문장화 한 뒤 토큰화
def expand_abbr(abbr_lst, abbr_dict):
    temp = []
    for abbr in abbr_lst:
        if abbr in abbr_dict:
            temp.extend(abbr_dict[abbr].split())
        else:
            if len(abbr) > 1:
                temp.append(abbr)
    return temp

# 벡터들의 평균 구하기
def mean_vector(vectors):
    return np.mean(vectors, axis=0)


def feature_engineering(data, pattern, abbr_dict):
    '''
    params
        data : DataFrame - feature engineering의 대상이 되는 데이터
    return
        DataFrame
    '''
    # ticketno column을 int64로 변경
    temp = deepcopy(data)
    temp['ticketno'] = temp['ticketno'].astype('int64')

    # ticketno, alarmno를 로그변환
    temp['ticketno_log1p'] = np.log1p(temp['ticketno'])
    temp['alarmno_log1p'] = np.log1p(temp['alarmno'])

    # alarmtime column을 datetime64로 변경
    tz_pytz = pytz.timezone('Asia/Seoul')
    temp['alarmtime'] = pd.to_datetime(temp['alarmtime'], unit='ns', utc=True).dt.tz_convert('Asia/Seoul')

    # alarmmsg
    temp["alarmmsg_vector"] = temp["alarmmsg_original"].str.lower()
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : re.sub(pattern, ' ', x))
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : re.findall('[A-Za-z0-9]+', x))
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : expand_abbr(x, abbr_dict))
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : model.wv[x])
    temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : mean_vector(x))
    # temp['alarmmsg_vector'] = temp['alarmmsg_vector'].apply(lambda x : np.expand_dims(x, axis=0))


    # alarmlevel 범주화
    temp['alarmlevel'] = temp['alarmlevel'].astype('object')

    # site & sysname을 알파벳 단위로 분리
    # site
    site_col_dict = {}
    for i in range(4):
        site_col_dict[f'site{i}'] = []

    for idx, value in temp['site'].items():
        for num, each_char in enumerate(value):
            site_col_dict[f'site{num}'].append(each_char)
    site_df = pd.DataFrame(site_col_dict)
    # sysname
    sysname_col_dict = {}
    for i in range(4):
        sysname_col_dict[f'sysname{i}'] = []

    for idx, value in temp['sysname'].items():
        for num, each_char in enumerate(value):
            sysname_col_dict[f'sysname{num}'].append(each_char)

    sysname_df = pd.DataFrame(sysname_col_dict)
    temp = pd.concat([temp, site_df, sysname_df], axis=1)

    # 필요한 변수들만 추출
    temp = temp[['alarmtime', 'alarmlevel', 'alarmmsg_vector', 'site1', 'sysname1',	'alarmno_log1p', 'ticketno_log1p']]

    # 다차원 벡터를 1차원의 변수들로 분해
    for i in range(options['vector_size']):
        temp[f'alarmmsg_vector_{i}'] = temp['alarmmsg_vector'].apply(lambda x: x[i])

    # 처리가 끝난 다차원 벡터를 제거
    temp.drop(columns=['alarmmsg_vector'], inplace=True)

    return temp

In [27]:
temp = feature_engineering(test, pattern, full_name)
temp

,alarmtime,alarmlevel,site1,sysname1,alarmno_log1p,ticketno_log1p,alarmmsg_vector_0,alarmmsg_vector_1,alarmmsg_vector_2,alarmmsg_vector_3,...,alarmmsg_vector_54,alarmmsg_vector_55,alarmmsg_vector_56,alarmmsg_vector_57,alarmmsg_vector_58,alarmmsg_vector_59,alarmmsg_vector_60,alarmmsg_vector_61,alarmmsg_vector_62,alarmmsg_vector_63
0,2022-12-25 00:02:16+09:00,5,E,f,28.144978,16.897989,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761
1,2022-12-25 00:02:51+09:00,5,D,e,28.144978,16.896318,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761
2,2022-12-25 00:03:22+09:00,4,E,f,28.144978,16.897065,0.000518,-0.000921,-0.001286,-0.000400,...,-0.000288,0.000421,-0.000915,-0.000214,-0.000591,0.000319,-0.000576,-0.000196,0.001479,0.002002
3,2022-12-25 00:03:33+09:00,5,C,c,28.144978,16.897990,-0.001725,-0.000297,-0.001454,0.000808,...,0.000107,-0.000048,-0.000095,0.000483,0.000663,-0.000269,0.001800,-0.001471,-0.001485,-0.001266
4,2022-12-25 00:03:39+09:00,5,E,f,28.144978,16.897990,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37666,2022-12-30 23:58:30+09:00,5,D,e,28.145288,16.905927,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761
37667,2022-12-30 23:58:33+09:00,4,E,f,28.145288,16.907247,0.000518,-0.000921,-0.001286,-0.000400,...,-0.000288,0.000421,-0.000915,-0.000214,-0.000591,0.000319,-0.000576,-0.000196,0.001479,0.002002
37668,2022-12-30 23:58:34+09:00,5,C,c,28.145288,16.905936,-0.001742,-0.000741,0.000223,0.000069,...,-0.000012,-0.000275,0.000147,-0.000378,-0.000830,-0.000010,0.000973,0.000158,0.001089,-0.001326
37669,2022-12-30 23:59:32+09:00,5,E,f,28.145288,16.907248,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761


# 타겟 예측 모델 불러오기(Autogluon)

In [22]:
predictor = TabularPredictor.load(save_path)

In [23]:
test_data = TabularDataset(temp)
test_data

,alarmtime,alarmlevel,site1,sysname1,alarmno_log1p,ticketno_log1p,alarmmsg_vector_0,alarmmsg_vector_1,alarmmsg_vector_2,alarmmsg_vector_3,...,alarmmsg_vector_54,alarmmsg_vector_55,alarmmsg_vector_56,alarmmsg_vector_57,alarmmsg_vector_58,alarmmsg_vector_59,alarmmsg_vector_60,alarmmsg_vector_61,alarmmsg_vector_62,alarmmsg_vector_63
0,2022-12-25 00:02:16+09:00,5,E,f,28.144978,16.897989,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761
1,2022-12-25 00:02:51+09:00,5,D,e,28.144978,16.896318,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761
2,2022-12-25 00:03:22+09:00,4,E,f,28.144978,16.897065,0.000518,-0.000921,-0.001286,-0.000400,...,-0.000288,0.000421,-0.000915,-0.000214,-0.000591,0.000319,-0.000576,-0.000196,0.001479,0.002002
3,2022-12-25 00:03:33+09:00,5,C,c,28.144978,16.897990,-0.001725,-0.000297,-0.001454,0.000808,...,0.000107,-0.000048,-0.000095,0.000483,0.000663,-0.000269,0.001800,-0.001471,-0.001485,-0.001266
4,2022-12-25 00:03:39+09:00,5,E,f,28.144978,16.897990,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37666,2022-12-30 23:58:30+09:00,5,D,e,28.145288,16.905927,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761
37667,2022-12-30 23:58:33+09:00,4,E,f,28.145288,16.907247,0.000518,-0.000921,-0.001286,-0.000400,...,-0.000288,0.000421,-0.000915,-0.000214,-0.000591,0.000319,-0.000576,-0.000196,0.001479,0.002002
37668,2022-12-30 23:58:34+09:00,5,C,c,28.145288,16.905936,-0.001742,-0.000741,0.000223,0.000069,...,-0.000012,-0.000275,0.000147,-0.000378,-0.000830,-0.000010,0.000973,0.000158,0.001089,-0.001326
37669,2022-12-30 23:59:32+09:00,5,E,f,28.145288,16.907248,-0.000876,0.000292,0.001746,0.002229,...,0.004287,0.001127,-0.001135,-0.002401,0.003845,-0.003692,0.000954,0.001978,-0.001393,-0.000761


# 예측하기

In [34]:
# Autogluon이 사용한 모든 모델에 대한 예측 결과 확인
for model_name in predictor.get_model_names():
    print(f'모델 이름 : {model_name}')
    pred = predictor.predict(test_data, model=model_name)
    test[target] = pred
    answer = test.groupby('ticketno', group_keys=True)['root_cause_type'].apply(lambda x : x.value_counts().index[0]).reset_index()
    unique_elements, counts_elements = np.unique(answer["root_cause_type"], return_counts=True)
    print("각 범주의 개수:", unique_elements, counts_elements)

모델 이름 : KNeighborsUnif
각 범주의 개수: ['LinkCut'] [4327]
모델 이름 : KNeighborsDist
각 범주의 개수: ['LinkCut' 'UnitFail'] [4248   79]
모델 이름 : NeuralNetFastAI
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [3895  110  322]
모델 이름 : LightGBMXT
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [4080  146  101]
모델 이름 : LightGBM
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [3950  269  108]
모델 이름 : RandomForestGini
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [4041  184  102]
모델 이름 : RandomForestEntr
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [4057  168  102]
모델 이름 : CatBoost
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [4078  149  100]
모델 이름 : ExtraTreesGini
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [3994  231  102]
모델 이름 : ExtraTreesEntr
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [4014  211  102]
모델 이름 : XGBoost
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [4067  146  114]
모델 이름 : NeuralNetTorch
각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [4123  105   99]
모델 이름 : LightGBMLarge
각 범주의 개수: ['LinkCut' 'Powe

In [35]:
# 원하는 모델로 예측하기
    # model에 원하는 모델의 이름 넣으면 된다.
model_name = 'NeuralNetFastAI'
prediction = predictor.predict(test_data, model=model_name)
unique_elements, counts_elements = np.unique(prediction, return_counts=True)
print("각 범주의 개수:", unique_elements, counts_elements)

각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [35989   773   909]


In [38]:
prediction = predictor.predict(test_data, model=model_name)
test[target] = prediction
answer = test.groupby('ticketno', group_keys=True)['root_cause_type'].apply(lambda x : x.value_counts().index[0]).reset_index()
unique_elements, counts_elements = np.unique(answer["root_cause_type"], return_counts=True)
print("각 범주의 개수:", unique_elements, counts_elements)

각 범주의 개수: ['LinkCut' 'PowerFail' 'UnitFail'] [3895  110  322]
